<a href="https://colab.research.google.com/github/artoowang/cython_colab/blob/main/Cython%2BC_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%pip install git+https://github.com/artoowang/cython_colab.git
import cython_colab

  Cloning https://github.com/artoowang/cython_colab.git to /tmp/pip-req-build-ccyy1y45
  Running command git clone -q https://github.com/artoowang/cython_colab.git /tmp/pip-req-build-ccyy1y45
  Created wheel for cython-colab: filename=cython_colab-0.1-py3-none-any.whl size=15328 sha256=442eb368e4ce7dfc8de28132cf013bf26c5146f16b120bd0692dbc04c5a85f0c
  Stored in directory: /tmp/pip-ephem-wheel-cache-fyldjwbj/wheels/b8/34/21/093444b60cd46f3574266da48cf6757ff776a72fddc548f846
Successfully built cython-colab


# Example

### Cython setup

In [2]:
%%file example_mod_setup.py

from distutils.core import setup  
from distutils.extension import Extension  
from Cython.Build import cythonize
import sys

# It is important to make sure the extension name, .pyx file, and this setup
# file names are all consistent with each other. E.g., to build a module called
# 'example_mod', the extension name should be 'example_mod', .pyx file should be
# example_mod.pyx, and the setup file should be example_mod_setup.py. These are
# used in various places in the module reload utility methods.
module_name = 'example_mod'

# OpenMP setup is platform dependent.
if sys.platform == 'darwin':
  sys_dep_compile_args = [
      '-Wno-unreachable-code',
      '-Wp,-fopenmp',
  ]
  sys_dep_link_args = [
      '-Wp,-fopenmp',
      '-lomp',
  ]
else:
  sys_dep_compile_args = ['-fopenmp']
  sys_dep_link_args = ['-fopenmp']

extensions = [
  Extension(module_name, [
                f'{module_name}.pyx',
                'example.cc',
            ],
            language='c++',
            extra_compile_args=[
                '-std=c++17',
            ] + sys_dep_compile_args,
            extra_link_args=sys_dep_link_args,
           ),
]

setup(ext_modules = cythonize(extensions, language_level='3'))

Writing example_mod_setup.py


### C++ sources

In [3]:
%%file example.h

#ifndef EXAMPLE_H
#define EXAMPLE_H

int AddOne(int val);

#endif  // EXAMPLE_H

Writing example.h


In [4]:
%%file example.cc

#include "example.h"

int AddOne(int val) {
  return val + 1;
}

Writing example.cc


### Cython sources

In [5]:
%%file example_mod.pyx

cimport cython
from cpython cimport array

from cython.parallel import prange


cdef extern from 'example.h':
  int AddOne(int val) nogil


@cython.boundscheck(False)
def run(Py_ssize_t length):
  cdef array.array buffer = array.array('i', [i for i in range(length)])
  cdef int[:] buffer_view = buffer
  cdef Py_ssize_t i
  for i in prange(length, nogil=True):
    buffer_view[i] = AddOne(buffer_view[i])
  return list(buffer)

Writing example_mod.pyx


### Build and test

In [6]:
example_mod = cython_colab.build_extension('example_mod')
example_mod.run(10)

Build extension example_mod ...
Next reload number: 1
Compiling example_mod.pyx because it changed.
[1/1] Cythonizing example_mod.pyx
running build_ext
building 'example_mod' extension
creating build
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c example_mod.cpp -o build/temp.linux-x86_64-3.7/example_mod.o -std=c++17 -fopenmp
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]